# Homework Week 2

Michael Brydon (mjbrydon@sfu.ca)

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import pymc3 as pm
import arviz as az

## Get data


In [27]:
d = pd.read_csv("../resources/Rethinking_2/Data/Howell1.csv", sep=";", header=0)
d.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041915,41.0,1
4,145.415,41.276872,51.0,0


### Filter out children

As before, we should take out children

In [246]:
d_adult = d[d["age"] >= 18]

## Create QUAP regression model

First, calculate the mean weight.  This is used for centering later.

In [247]:
weight_avg = d_adult["weight"].mean()

Weight is defined as data here ("shared variable" [here](https://docs.pymc.io/notebooks/posterior_predictive.html)).  This allows us to swap out observed values for weights with out-of-sample values later.

In [273]:
with pm.Model() as mod1:
    sigma = pm.Uniform("sigma", 0, 50)
    alpha = pm.Normal("alpha", 178, 20)
    beta = pm.Lognormal("beta", 0 , 1)
    weight = pm.Data("weight", d_adult["weight"])
    mu = alpha + beta * (weight - weight_avg)
    height = pm.Normal("height", mu, sigma, observed = d_adult["height"])

In [274]:
map_estimate = pm.find_MAP(model=mod1)
map_estimate

{'sigma_interval__': array(-2.18135226),
 'alpha': array(154.60136748),
 'beta_log__': array(-0.10172172),
 'sigma': array(5.07188029),
 'beta': array(0.90328088)}

## PyMC3 approach

PyMC3 folks don't like quadratic approximation, so they run a MCMC sample:

In [291]:
with mod1:
    trace = pm.sample(2000, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 13 seconds.


In [292]:
with mod1:
    mod1sum = az.summary(trace, round_to=2, kind="stats")
mod1sum

,mean,sd,hdi_3%,hdi_97%
alpha,154.6,0.27,154.09,155.10
sigma,5.1,0.20,4.74,5.47
beta,0.9,0.04,0.82,0.99


## Create a dataframe for out-of-sample data

In [293]:
new = pd.DataFrame({"weight": [45, 40, 65, 31]})

As far as I can tell, the `set_data()` function needs a dictionary. So the step above is reversed.

In [294]:
weights_dict = {"weight": new["weight"].values}

In [295]:
with mod2:
    pm.set_data(weights_dict)
    h_sim = pm.sample_posterior_predictive(trace)

In [296]:
new["exp_height"] = h_sim["height"].mean(0) # 0-axis for means by column

Or this form, which is more consistent with the approach for the PIs:

In [297]:
np.mean(preds["height"], 0)

array([154.46089481, 150.11201569, 172.63817766, 141.94344555])

In [298]:
new["LPI(5.5)"] = np.percentile(preds["height"], 5.5, axis=0)
new["UPI(94.5)"] = np.percentile(preds["height"], 94.5, axis=0)

In [299]:
new.round(1)

,weight,exp_height,LPI(5.5),UPI(94.5)
0,45,154.4,146.2,162.7
1,40,150.3,141.9,158.2
2,65,172.7,164.5,181.1
3,31,142.0,133.7,150.0
